<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Шаг-1.-Откройте-файл-с-данными-и-изучите-общую-информацию" data-toc-modified-id="Шаг-1.-Откройте-файл-с-данными-и-изучите-общую-информацию-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Шаг 1. Откройте файл с данными и изучите общую информацию</a></span></li><li><span><a href="#Шаг-2.-Предобработка-данных" data-toc-modified-id="Шаг-2.-Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Шаг 2. Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Шаг-3.-Ответьте-на-вопросы" data-toc-modified-id="Шаг-3.-Ответьте-на-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Шаг 3. Ответьте на вопросы</a></span></li><li><span><a href="#Шаг-4.-Общий-вывод" data-toc-modified-id="Шаг-4.-Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Шаг 4. Общий вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [803]:
import pandas as pd

from pymystem3 import Mystem
from collections import Counter

In [760]:
try:
    data = pd.read_csv('/datasets/data.csv')
except FileNotFoundError:
    data = pd.read_csv('data.csv')

data.info()
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Первичный анализ показывает, что в данных имеются проблемы, которые требуют предобработки:
- пропуски (столбцы `days_employed` и `total_income`)
- ошибки в данных (`days_employed` содержит вещественные числа, хотя по описанию должны быть целые, кроме того встречаются как отрицательные, так и положительные значения)
- есть неявные дубликаты (`education` содержит одинаковые строки в разном регистре)

Требуется более глубокий анализ, чтобы проверить наличие других ошибок в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Согласно первоначальному анализу, пропуски есть в стобцах `day_employed` и `total_income`. Кроме того, количество пропусков в обоих столбцах совпадает. Проверим гипотезу, что пропуски наблюдаются одновременно в обоих столбцах.

In [761]:
print(f'Количество пропусков в столце "days_emploeyed": {len(data[data["days_employed"].isna()])}')
print(f'Количество пропусков в столце "total_income": {len(data[data["total_income"].isna()])}')
print(f"Количество пропусков одновременно в обоих столбцах:", end=' ')
print(len(data[data['days_employed'].isna() & data['total_income'].isna()]))


Количество пропусков в столце "days_emploeyed": 2174
Количество пропусков в столце "total_income": 2174
Количество пропусков одновременно в обоих столбцах: 2174


Действительно, гипотеза подтвердилась и пропуски наблюдаются одновременно в обоих стобцах. Вероятнее всего пропуски связаны со сбоем в выгрузке данных.

Посмотрим внимательно на столбец `days_emploed`. Из описания таблицы следует, что здесь записан трудовой стаж в днях.

In [762]:
data['days_employed'].head(20)

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
15      -972.364419
16     -1719.934226
17     -2369.999720
18    400281.136913
19    -10038.818549
Name: days_employed, dtype: float64

В столбце крайне странные значения. Во-первых, тип данных `float`, стоило ожидать, что трудовой стаж в днях должен быть целым положительным числом. Во-вторых, встречаются отрицательные числа в большом количестве. В-третьих, положительные числа очень большие например в строке с индексом 4 указан стаж более 340 тысяч дней, а это более 900 лет.
Что ж, кажется данные в столбце `days_employed` полностью некачественные и ими нельзя пользоваться. Кроме того, в нашем исследовании данные о трудовом стаже не требуются. Поэтому, думаю, мы может просто избавиться от этого стобца.

In [763]:
data = data.drop(columns=['days_employed'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


Теперь разберемся со стобцом `total_income`, в котором хранятся данные о доходе в месяц.

In [764]:
display(data['total_income'].head(10))
print('Минимальный доход:', data['total_income'].min())
print('Максимальный доход:', data['total_income'].max())

0    253875.639453
1    112080.014102
2    145885.952297
3    267628.550329
4    158616.077870
5    255763.565419
6    240525.971920
7    135823.934197
8     95856.832424
9    144425.938277
Name: total_income, dtype: float64

Минимальный доход: 20667.26379327158
Максимальный доход: 2265604.028722744


Наверно немного смущает сам размер месячного дохода, в России редко увидишь такие зарплаты. Но вообще у нас нет данных о том, о какой стране и валюте идет речь.
Проблема заполнения пропусков типичным значением (средним арифмитическим или медианой) заключается в том, что один из заданных нам вопросов связан с влиянием дохода на возврат кредита. Объем пропущенных значений - около 10% от данных. С другой стороны, терять 10% данных, удалив эти строки, тоже не хочется, потому что в других столбцах этих строк данные полные. Применим такой подход - заполним пропущенные значения нулем. Так мы сможем при анализе зависимости возврата кредита от дохода отфильтровать значения, которые нам неизвестны. При этом для анализа других столбцов у нас будет больше данных.

In [765]:
data['total_income'] = data['total_income'].fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


Мы увидели, что столбец `day_employed` полностью состоял из "грязных данных". Посмотрим, есть ли похожие проблемы в других колонках.

In [766]:
for column in data:
    if column != 'total_income':
        print(f'Уникальные значения в стобце "{column}":')
        print(data[column].sort_values().unique())
        print()

Уникальные значения в стобце "children":
[-1  0  1  2  3  4  5 20]

Уникальные значения в стобце "dob_years":
[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]

Уникальные значения в стобце "education":
['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']

Уникальные значения в стобце "education_id":
[0 1 2 3 4]

Уникальные значения в стобце "family_status":
['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']

Уникальные значения в стобце "family_status_id":
[0 1 2 3 4]

Уникальные значения в стобце "gender":
['F' 'M' 'XNA']

Уникальные значения в стобце "income_type":
['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудни

Очень странными выглядят значения -1 и 20 в стобце `children`, посмотрим на эти строки.

In [767]:
data[(data['children'] == -1) | (data['children'] == 20)]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
606,20,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
705,-1,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
720,20,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
742,-1,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
...,...,...,...,...,...,...,...,...,...,...,...
21140,-1,44,среднее,1,женат / замужем,0,F,компаньон,0,169562.091999,операции со своей недвижимостью
21325,20,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,53,среднее,1,женат / замужем,0,M,компаньон,0,0.000000,покупка жилой недвижимости
21404,20,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


Таких строк оказалось 123. Почти наверняка ошибка - человеский фактор. Вероятнее всего нам следует -1 изменить на 1, а у 20 убрать 0, сделав 2:

In [768]:
data['children'].replace(20, 2, inplace=True)
data['children'].replace(-1, 1, inplace=True)
print(f'Уникальные значения "children":', *sorted(data["children"].unique()))

Уникальные значения "children": 0 1 2 3 4 5


С детьми разобрались, двигаемся дальше.

Есть явно ошибочное значение 0 в графе возраста заемщиков `dob_years`.

In [769]:
print("Количество строк с возрастом заемщика 0:", len(data[data['dob_years'] == 0]))

Количество строк с возрастом заемщика 0: 101


Нашлась 101 строка. Не так уж много на выборке более 21 тысячи.
Тем не менее, чтобы не терять эти данные, которые в остальном кажутся нормальными, поменяем значение 0 на медианное.

In [770]:
dob_years_median = int(data[data['dob_years'] != 0]['dob_years'].median())
print(f'Медианное значение возраста: {dob_years_median}')

Медианное значение возраста: 43


In [771]:
data['dob_years'].replace(0, 43, inplace=True)

Переходим к стобцу `education`.
Уникальные значения в стобце "education":
['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']

Здесь одинаковые типы уровня образования записаны в разном регистре. Лучшее решение - привести все буквы к строчным.

In [772]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

В стобце `gender` мы встретили странное значение `XNA` помимо ожидаемыеx `M` и `F`. Посмотрим на их количество.

In [773]:
data[data['gender'] == 'XNA']


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Что ж, всего одно значение. Похоже, что кто-то просто забыл указать пол. Мы можем просто удалить строку, 1 анкета никак не повлияет на нашу большую выборку, но можем и оставить, так как нас не требуется узнать влияние пола на возврат кредита.

Проблему с пропусками решили, попутно еще исправив некоторые некорректные данные.

### Замена типа данных

Теперь заменим тип данных в `total_income` с `float` на `int`. Для этой цели подойдет метод `astype()`

In [774]:
data['total_income'] = data['total_income'].astype('int')

Проверяем:

In [775]:
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Мы использовали разные способы для борьбы с пропусками. Мы смогли избежать удаления строк с пропусками, что могло бы сказаться в дальнейшем на результатах исследования. В первом случае мы избавились от столбца, в котором были явно ошибочные данные. Для стобца, показывающего доход в месяц, мы заполнили пропущенные значения медианным значением, так как это наилучший способ для количественных значений в выборке с выбросами.

### Обработка дубликатов

Найдем явные дубликаты, посмотрим количество строк, которые полностью совпадают.

In [776]:
print('Количество явных дубликатов:', data.duplicated().sum())


Количество явных дубликатов: 71


Избавимся от этих строк. Скорее всего при внесении данных были продублированы 71 строка, потому что кажется очень маловероятным, чтобы совпадали все поля, вплоть до `total_income`

In [777]:
data = data.drop_duplicates().reset_index(drop=True)
print('Количество явных дубликатов:', data.duplicated().sum())

Количество явных дубликатов: 0


In [778]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 11 columns):
children            21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


**Вывод**

Мы избавились от явных дубликатов, и в том числе, потому что ранее привели к одному регистру данные из столбца `education`.

### Лемматизация

Посмотрим еще раз, какие указывались цели кредита:

In [779]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [780]:
unique_purpose = data['purpose'].unique()
print('Количество уникальных значений "purpose":', len(unique_purpose))

Количество уникальных значений "purpose": 38


На первый взгляд очевидно, что цели кредита, например "на покупку автомобиля" и "свой автомобиль" можно объединить в более крупные подгруппы.
Посчитаем, применив лемматизацию, какие слова в стобце `purpose` встречаются чаще всего

In [802]:
all_purposes = ' '.join(unique_purpose)
m = Mystem()
lemmas = m.lemmatize(all_purposes)
print(Counter(lemmas))


Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Изучив частоту встречающихся слов,  логично выделить следующие подгруппы:
- свадьба
- ипотека ("недвижимость" / "жилье")
- образование
- автомобиль

Любое из значений столбца `purpose` можно отнести к одной из этих 4 подгрупп.

Сначала напишем функцию `get_purpose_group`, принимающую значение `purpose` в качестве аргумента и возвращающую одно из 4 значений подгруппы целей.

In [781]:
def get_purpose_group(purpose: str) -> str:
    lemma_purpose = m.lemmatize(purpose)
    if 'свадьба' in lemma_purpose:
        return 'свадьба'
    elif 'образование' in lemma_purpose:
        return 'образование'
    elif 'автомобиль' in lemma_purpose:
        return 'автомобиль'
    elif 'жилье' in lemma_purpose or 'недвижимость' in lemma_purpose:
        return 'ипотека'
    else:
        return purpose


Теперь изменим столбец `purpose` и проверим, что там у нас получилось 4 подгруппы:

In [782]:
data['purpose'] = data['purpose'].apply(get_purpose_group)
data['purpose'].value_counts()

ипотека        10811
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose, dtype: int64

После того, как мы обобщили цели кредита, проверим еще раз наши данные на предмет дубликатов

In [783]:
print('Количество дубликатов:', data.duplicated().sum())

Количество дубликатов: 337


Довольно интересно, у нас обнаружилось еще 337 дубликатов. Учитывая, что случайное полное совпадение, ввиду наличия вещественного столбца в данных, очень маловероятно, то скорее можно предположить, что дубликаты возникли от того, что несколько человек подавали заявление на кредит больше одного раза, указывая чуть иную формулировку для цели кредита.
От таких дубликатов логично избавиться.

In [784]:
data = data.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов:', data.duplicated().sum())

Количество дубликатов: 0


**Вывод**

Благодаря лемматизации данных столбца `purpose` нам удалось, во-первых, выделить 4 большие подгруппы целей кредита, вместо изначальных 38, а во-вторых, найти и удалить еще 337 дубликатов.

### Категоризация данных

Еще раз взглянем на нашу таблицу с данными:

In [785]:
data

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,ипотека
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,ипотека
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...,...
21112,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,ипотека
21113,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль
21114,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,ипотека
21115,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль


Для того, чтобы ответить на один из вопросов ("Влияет ли наличие детей на возврат кредита") добавим столбец `has_children`

In [786]:
def has_children(num_children: int) -> bool:
    return num_children != 0

data['has_children'] = data['children'].apply(has_children)
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,has_children
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,ипотека,True
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,True
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,ипотека,False
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,True
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,False


Семейное положение и образование уже продублированы своим id. Длинные наименования делают нашу таблицу объемной и трудночитаемой. Поэтому выделим отдельные словари для них и в таблице оставим только идентификаторы, а наименования уберем.

In [787]:
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
display(education_dict)

family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
display(family_status_dict)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [788]:
data.drop(columns=['education', 'family_status'], inplace=True)
data.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,has_children
0,1,42,0,0,F,сотрудник,0,253875,ипотека,True
1,1,36,1,0,F,сотрудник,0,112080,автомобиль,True
2,0,33,1,0,M,сотрудник,0,145885,ипотека,False
3,3,32,1,0,M,сотрудник,0,267628,образование,True
4,0,53,1,1,F,пенсионер,0,158616,свадьба,False


Теперь нам нужно категоризировать данные об уровне дохода. Как мы помним, у нас были данные, где уровень дохода был неизвестен.
С помощью функции  `decribe()` найдем квартили, сделаем массив квартилей и напишем функцию, которая на основании `total_income` в зависимости от уровня дохода распределит в новый столбец  `income_level`:
- 0 - нет данных
- 0-25 - низкий
- 25-50 - ниже среднего
- 50-75 - выше среднего
- \> 75 - высокий

In [789]:
income_description = data[data['total_income'] != 0]['total_income'].describe()
income_description

count    1.935100e+04
mean     1.674218e+05
std      1.029716e+05
min      2.066700e+04
25%      1.030530e+05
50%      1.450170e+05
75%      2.034345e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [790]:
quantiles = [income_description['25%'], income_description['50%'], income_description['75%']]

def get_income_level(income: int) -> str:
    if income == 0:
        return 'нет данных'
    elif 0 < income <= income_description['25%']:
        return 'низкий'
    elif income_description['25%'] < income <= income_description['50%']:
        return 'ниже среднего'
    elif income_description['50%'] < income <= income_description['75%']:
        return 'выше среднего'
    else:
        return 'высокий'

data['income_level'] = data['total_income'].apply(get_income_level)
data.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,has_children,income_level
0,1,42,0,0,F,сотрудник,0,253875,ипотека,True,высокий
1,1,36,1,0,F,сотрудник,0,112080,автомобиль,True,ниже среднего
2,0,33,1,0,M,сотрудник,0,145885,ипотека,False,выше среднего
3,3,32,1,0,M,сотрудник,0,267628,образование,True,высокий
4,0,53,1,1,F,пенсионер,0,158616,свадьба,False,выше среднего


И наконец сделаем таблицу, в которой соберем все необходимые для дальнейшего анализа данные.

In [791]:
research_data = data[['debt', 'has_children', 'family_status_id', 'income_level', 'purpose']]
research_data.info()
research_data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21117 entries, 0 to 21116
Data columns (total 5 columns):
debt                21117 non-null int64
has_children        21117 non-null bool
family_status_id    21117 non-null int64
income_level        21117 non-null object
purpose             21117 non-null object
dtypes: bool(1), int64(2), object(2)
memory usage: 680.7+ KB


,debt,has_children,family_status_id,income_level,purpose
0,0,True,0,высокий,ипотека
1,0,True,0,ниже среднего,автомобиль
2,0,False,0,выше среднего,ипотека
3,0,True,0,высокий,образование
4,0,False,1,выше среднего,свадьба
5,0,False,1,высокий,ипотека
6,0,False,0,высокий,ипотека
7,0,False,0,ниже среднего,образование
8,0,True,1,низкий,свадьба
9,0,False,0,ниже среднего,ипотека


**Вывод**

Мы выделили нужные нам для анализа данные и  объединили их в подходящие категории

## Шаг 3. Ответьте на вопросы

Для начала будет интересно найти среднее значение возврата кредита

In [792]:
avg_debt = research_data['debt'].sum() / len(research_data)
print(f'Средний процент возврата кредита в срок: {1 - avg_debt: .1%}')

Средний процент возврата кредита в срок:  91.8%


- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [793]:
has_children_debt = research_data.pivot_table(index='has_children', values='debt')
has_children_debt

,debt
has_children,
False,0.076734
True,0.093004


In [794]:
print('Процент возврата кредита')
print(f"* есть дети: {1 - has_children_debt.loc[True, 'debt']:.1%}")
print(f"* нет детей: {1 - has_children_debt.loc[False, 'debt']:.1%}")

Процент возврата кредита
* есть дети: 90.7%
* нет детей: 92.3%


**Вывод**

Наличие детей ухудшает возврат кредита на 1.1%, а отсутствие - увеличивает на 0.5% (относительно среднего значения)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [795]:
family_status_debt = research_data.pivot_table(index='family_status_id', values='debt')
family_status_debt = family_status_dict.join(family_status_debt, on='family_status_id').sort_values(by='debt').reset_index(drop=True)
family_status_debt[['family_status', 'debt']]

,family_status,debt
0,вдовец / вдова,0.066808
1,в разводе,0.071249
2,женат / замужем,0.076942
3,гражданский брак,0.094106
4,Не женат / не замужем,0.098420


In [796]:
print('Процент возврата кредита в зависимости от семейного положения')
print('____________')
for row in family_status_debt.itertuples():
    print(f'{row[2]:<30}: {1 - row[3]:.1%}')

Процент возврата кредита в зависимости от семейного положения
____________
вдовец / вдова                : 93.3%
в разводе                     : 92.9%
женат / замужем               : 92.3%
гражданский брак              : 90.6%
Не женат / не замужем         : 90.2%


**Вывод**

Исследование показало, что лучше всего возвращают кредит люди, у которых в прошлом был опыт семейной жизни - потерявшие вторую половину из-за смерти, а так же разведенные. Чуть хуже возвращают кредит те, кто находится в браке официально. И чаще всего проблемы с возвратом у банков возникают с теми, у кого нет печатей в паспорте.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [797]:
income_level_debt = research_data.pivot_table(index='income_level', values='debt')
income_level_debt = income_level_debt.sort_values(by='debt').reset_index()
income_level_debt

,income_level,debt
0,высокий,0.070484
1,низкий,0.079165
2,ниже среднего,0.087019
3,выше среднего,0.088071
4,нет данных,0.095130


In [798]:
print('Процент возврата кредита в зависимости от уровня дохода')
print('____________')
for row in income_level_debt.itertuples():
    print(f'{row[1]:<30}: {1 - row[2]:.1%}')

Процент возврата кредита в зависимости от уровня дохода
____________
высокий                       : 93.0%
низкий                        : 92.1%
ниже среднего                 : 91.3%
выше среднего                 : 91.2%
нет данных                    : 90.5%


**Вывод**

Лучше всего возвращают кредит люди с высоким уровнем дохода, что выглядит логичным. И неожиданно, что люди с низким доходом возвращают кредит лучше среднего показателя. Кроме того, кажется мы поступили правильно, сделав отдельную категорию для записей, где изначально не был указан доход. Вполне возможно, что это совсем не случайность и доход не был указан вполне осознанно.

- Как разные цели кредита влияют на его возврат в срок?

In [799]:
purpose_debt = research_data.pivot_table(index='purpose', values='debt')
purpose_debt = purpose_debt.sort_values(by='debt').reset_index()
purpose_debt

,purpose,debt
0,ипотека,0.073853
1,свадьба,0.080659
2,образование,0.093340
3,автомобиль,0.094101


In [800]:
print('Процент возврата кредита в зависимости от его цели')
print('____________')
for row in purpose_debt.itertuples():
    print(f'{row[1]:<30}: {1 - row[2]:.1%}')

Процент возврата кредита в зависимости от его цели
____________
ипотека                       : 92.6%
свадьба                       : 91.9%
образование                   : 90.7%
автомобиль                    : 90.6%


**Вывод**

Вполне ожидаемо, что просят ипотеку люди, у которых в прошлом не было проблем с выплатой кредитов. Потому что ипотечный кредит как правило самый крупный и заемщика будут рассматривать с особой тщательностью.

## Шаг 4. Общий вывод

Средний процент людей, подавших заявку на кредит, у которых в прошлом не было проблем с его возвратом - 91.8%
Самый надежный заемщик для банка, исходя из наших данных - вдова / вдовец без детей с высоким уровнем дохода, который берет ипотеку. Посмотрим, как много людей, которые подходят под все эти параметры

In [801]:
best_for_loan = (research_data[
    (research_data['has_children'] == False) &
    (research_data['income_level'] == 'высокий') &
    (research_data['family_status_id'] == 2) &
    (research_data['purpose'] == 'ипотека')
])
print(f"Процент возврата кредита у лучшей категории заемщиков {1 - best_for_loan['debt'].sum() / len(best_for_loan):.1%}")

Процент возврата кредита у лучшей категории заемщиков 97.3%


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.